In [229]:
# Essential Imports
import pandas as pd
import numpy as np

# Pipeline for Scaling, Feature selection, and Regression
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif, SelectFromModel
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer

# Metrics for Model
from sklearn.metrics import accuracy_score

### Examine Data to determine what needs to be cleaned

In [200]:
df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


### Convert categorical data to 0 and 1 for regression and drop any NA values before feature selection (will be added back afterwards)

In [201]:
data_dummies = pd.get_dummies(df_train,dummy_na=True)
imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
imputed_data = imputer.fit_transform(data_dummies)
data = pd.DataFrame(data=imputed_data, columns=imputer.get_feature_names_out()).drop(["Id","SalePrice"],axis=1)
data.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_Oth,SaleType_WD,SaleType_nan,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SaleCondition_nan
0,60.0,65.0,8450.0,7.0,5.0,2003.0,2003.0,196.0,706.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,20.0,80.0,9600.0,6.0,8.0,1976.0,1976.0,0.0,978.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,60.0,68.0,11250.0,7.0,5.0,2001.0,2002.0,162.0,486.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,70.0,60.0,9550.0,7.0,5.0,1915.0,1970.0,0.0,216.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,60.0,84.0,14260.0,8.0,5.0,2000.0,2000.0,350.0,655.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


### Perform scaling and feature selection using an sklearn pipeline

In [202]:
# Scale data
scaler_data = StandardScaler()
scaler_data.fit(data)
data_scaled = scaler_data.transform(data)

scaler_y = StandardScaler()
scaler_y.fit(data_dummies[["SalePrice"]])
y_scaled = scaler_y.transform(data_dummies[["SalePrice"]])

scaler_data.get_feature_names_out()
x_scaled = pd.DataFrame(data=data_scaled,columns=scaler_data.get_feature_names_out())
x_scaled.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_Oth,SaleType_WD,SaleType_nan,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SaleCondition_nan
0,0.073375,-0.229372,-0.207142,0.651479,-0.517200,1.050994,0.878668,0.511418,0.575425,-0.288653,...,-0.045376,0.390293,0.0,-0.272616,-0.052414,-0.091035,-0.117851,0.467651,-0.305995,0.0
1,-0.872563,0.451936,-0.091886,-0.071836,2.179628,0.156734,-0.429577,-0.574410,1.171992,-0.288653,...,-0.045376,0.390293,0.0,-0.272616,-0.052414,-0.091035,-0.117851,0.467651,-0.305995,0.0
2,0.073375,-0.093110,0.073480,0.651479,-0.517200,0.984752,0.830215,0.323060,0.092907,-0.288653,...,-0.045376,0.390293,0.0,-0.272616,-0.052414,-0.091035,-0.117851,0.467651,-0.305995,0.0
3,0.309859,-0.456474,-0.096897,0.651479,-0.517200,-1.863632,-0.720298,-0.574410,-0.499274,-0.288653,...,-0.045376,0.390293,0.0,3.668167,-0.052414,-0.091035,-0.117851,-2.138345,-0.305995,0.0
4,0.073375,0.633618,0.375148,1.374795,-0.517200,0.951632,0.733308,1.364570,0.463568,-0.288653,...,-0.045376,0.390293,0.0,-0.272616,-0.052414,-0.091035,-0.117851,0.467651,-0.305995,0.0


In [225]:
feature_selector = SelectKBest(score_func=f_classif, k=100)
x_new = feature_selector.fit_transform(x_scaled,y_scaled)

c:\Users\zhatz\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\zhatz\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [ 41  44  52  57  60  66  70  96 106 113 115 121 130 137 146 154 162 172
 179 189 193 195 202 232 238 244 247 258 266 299 316 323 330] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\zhatz\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
c:\Users\zhatz\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:

### Scale the test data based on teh training data

In [226]:
test_dummies = pd.get_dummies(df_test,dummy_na=True)
test_imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
imputed_test_data = test_imputer.fit_transform(test_dummies)
test = pd.DataFrame(data=imputed_test_data, columns=test_imputer.get_feature_names_out()).drop(["Id"],axis=1)
inclusion_columns = set(feature_selector.get_feature_names_out()) & set(test.columns)

new_scaler = StandardScaler()
new_scaler.fit(data.filter(items=inclusion_columns))
new_data = new_scaler.transform(data.filter(items=inclusion_columns))
x_test = new_scaler.transform(test.filter(items=inclusion_columns))

### Linear Regression

In [227]:
regressor = LinearRegression()
regressor.fit(X=new_data,y=y_scaled)
predicted = regressor.predict(X=x_test)

In [233]:
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10], "degree":[2,5]}
regressor_svm = svm.SVR()
cv_regressor = GridSearchCV(regressor_svm, parameters)
cv_regressor.fit(X=new_data,y=y_scaled)

c:\Users\zhatz\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\zhatz\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\zhatz\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\zhatz\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1184: DataC

GridSearchCV(estimator=SVR(),
             param_grid={'C': [1, 10], 'degree': [2, 5],
                         'kernel': ('linear', 'rbf')})

In [236]:
cv_regressor.best_score_

0.810036362133847

In [237]:
regressor_svm = svm.SVR(C=1, degree=2, kernel='linear')
regressor_svm.fit(X=new_data,y=y_scaled)
predicted = regressor_svm.predict(X=x_test)

c:\Users\zhatz\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [238]:
regressor_svm.score(X=new_data,y=y_scaled)

0.8508567976936299

In [216]:
pd.Series(scaler_y.inverse_transform(predicted).flatten())

0       114647.814924
1       175618.231069
2       163178.194957
3       182112.606553
4       199881.517613
            ...      
1454     91466.263388
1455    101827.047411
1456    149384.935505
1457    113351.643625
1458    211836.008791
Length: 1459, dtype: float64